In [1]:
## Important libraries
import streamlit as st
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
import json
from typing import List, Literal, Optional
import chromadb
import uuid
from transformers import AutoModel, AutoTokenizer
from langchain_core.messages import get_buffer_string
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import re
import tiktoken
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain_core.tools import tool
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

In [2]:
load_dotenv()

True

In [51]:
from langchain_community.vectorstores import Chroma 

In [3]:
llm=ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="deepseek-r1-distill-llama-70b"
)

In [4]:
res = llm.invoke("Hi")
res.content

'<think>\n\n</think>\n\nHello! How can I assist you today? 😊'

In [5]:
model_name = "BAAI/bge-small-en-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

C:\Users\dheeraj\AppData\Local\Temp\ipykernel_1928\791078931.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")


In [6]:
from langchain.prompts import ChatPromptTemplate

# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Use the provided context to answer {name}, {date} questions accurately."),
        ("placeholder", "{query_with_context}")
    ]
)

# Fill the template with actual values
prmp = prompt.invoke({
    "name": "Dheeraj",
    "date": "2024-03-20",
    "query_with_context": [("What is the weather like today in New Delhi?")]
})

# Print the formatted prompt
print(prmp)


messages=[SystemMessage(content='You are a helpful assistant. Use the provided context to answer Dheeraj, 2024-03-20 questions accurately.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the weather like today in New Delhi?', additional_kwargs={}, response_metadata={})]


In [1]:
from langchain_community.vectorstores import Chroma

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings


In [7]:
text = "Tokenization is important in NLP!"
tokens = tokenizer.tokenize(text)
tokens

['token', '##ization', 'is', 'important', 'in', 'nl', '##p', '!']

In [8]:
token_ids = tokenizer.encode("TOM")
token_ids

[101, 3419, 102]

In [9]:
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

In [10]:
inputs

{'input_ids': tensor([[  101, 19204,  3989,  2003,  2590,  1999, 17953,  2361,   999,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [11]:
output = model(**inputs)
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.5625, -0.2248,  0.2079,  ..., -0.1317,  0.5404, -0.2481],
         [-0.2984, -0.0176,  0.0511,  ..., -0.3413,  1.1455, -1.1604],
         [-0.4177, -0.2039,  0.3929,  ..., -0.2503,  0.9900,  0.0939],
         ...,
         [-0.9699, -0.3478,  0.2634,  ...,  0.1652,  0.7106, -0.2993],
         [-0.5622, -0.0389,  0.2668,  ..., -0.5186,  1.0669, -0.4123],
         [-0.5625, -0.2248,  0.2079,  ..., -0.1317,  0.5404, -0.2481]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.1013, -0.0475,  0.1447, -0.0351, -0.1429, -0.0728, -0.0626,  0.1253,
          0.0631, -0.0427,  0.0023,  0.0215, -0.0063, -0.0172, -0.0513,  0.1690,
          0.0357,  0.0085,  0.1014,  0.1401, -0.0222,  0.0570,  0.1195, -0.1327,
         -0.0566,  0.0637, -0.0355,  0.0064, -0.0809,  0.0585,  0.0251,  0.0185,
         -0.1385,  0.0537,  0.0637, -0.0616, -0.0306, -0.0131, -0.0147,  0.1220,
         -0.0863, -0.0379,  0.04

In [12]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is Carl."),
    ("human", "{user_input}"),
])
prompt_value = template.format(user_input="Who are you?")
prompt_value

'System: You are a helpful AI bot. Your name is Carl.\nHuman: Who are you?'

In [13]:
response=llm.invoke(prompt_value)
response

AIMessage(content='<think>\nOkay, so I\'m trying to figure out how to respond to the user\'s question. They asked, "Who are you?" and the system provided a response where the AI introduces itself as Carl, a helpful AI bot. Now, I need to think through how to approach this.\n\nFirst, I should consider the user\'s intent. They\'re asking for identification, so the response should clearly state what the AI is. The system\'s response does that by saying, "I\'m Carl, a helpful AI bot..." which is good. It\'s concise and informative.\n\nI wonder if there\'s anything else the user might want to know. Maybe they\'re curious about the AI\'s capabilities or limitations. The response mentions assisting with questions and providing information, which covers that. It also reassures the user that the AI is here to help, which is positive.\n\nIs there a way to make the response more engaging? Perhaps by adding a friendly emoji or a question to prompt further interaction. However, the current response

In [14]:
# Model for generating Embedding
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [15]:
recall_vector_store = InMemoryVectorStore(embedding_model)

In [16]:
def get_user_id(config: RunnableConfig) -> str:
    user_id = config["configurable"].get("user_id")
    if user_id is None:
        raise ValueError("User ID needs to be provided to save a memory.")

    return user_id

@tool
def save_recall_memory(memory: str, config: RunnableConfig) -> str:
    """Save memory to vectorstore for later semantic retrieval."""
    user_id = get_user_id(config)
    document = Document(
        page_content=memory, id=str(uuid.uuid4()), metadata={"user_id": user_id}
    )
    recall_vector_store.add_documents([document])
    return memory

@tool
def search_recall_memories(query: str, config: RunnableConfig) -> List[str]:
    """Search for relevant memories."""
    user_id = get_user_id(config)

    def _filter_function(doc: Document) -> bool:
        return doc.metadata.get("user_id") == user_id

    documents = recall_vector_store.similarity_search(
        query, k=3, filter=_filter_function
    )
    return [document.page_content for document in documents]

In [17]:
tools = [save_recall_memory, search_recall_memories]

In [18]:
class State(MessagesState):
    # add memories that will be retrieved based on the conversation context
    recall_memories: List[str]

In [19]:
# Define the prompt template for the agent
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant with advanced long-term memory"
            " capabilities. Powered by a stateless LLM, you must rely on"
            " external memory to store information between conversations."
            " Utilize the available memory tools to store and retrieve"
            " important details that will help you better attend to the user's"
            " needs and understand their context.\n\n"
            ## Rag Prompt
            " You are designed to help user in finding the best possible answer from uploased pdf"
            " files. You can ask user for clarification if you are unsure about"
            " Use the following context while answering the user's question:\n\n"
            " Context :\n {context}\n\n"
            ##########
            "Memory Usage Guidelines:\n"
            "1. Actively use memory tools (save_core_memory, save_recall_memory)"
            " to build a comprehensive understanding of the user.\n"
            "2. Make informed suppositions and extrapolations based on stored"
            " memories.\n"
            "3. Regularly reflect on past interactions to identify patterns and"
            " preferences.\n"
            "4. Update your mental model of the user with each new piece of"
            " information.\n"
            "5. Cross-reference new information with existing memories for"
            " consistency.\n"
            "6. Prioritize storing emotional context and personal values"
            " alongside facts.\n"
            "7. Use memory to anticipate needs and tailor responses to the"
            " user's style.\n"
            "8. Recognize and acknowledge changes in the user's situation or"
            " perspectives over time.\n"
            "9. Leverage memories to provide personalized examples and"
            " analogies.\n"
            "10. Recall past challenges or successes to inform current"
            " problem-solving.\n\n"
            "## Recall Memories\n"
            "Recall memories are contextually retrieved based on the current"
            " conversation:\n{recall_memories}\n\n"
            "## Instructions\n"
            "Engage with the user naturally, as a trusted colleague or friend."
            " There's no need to explicitly mention your memory capabilities."
            " Instead, seamlessly incorporate your understanding of the user"
            " into your responses. Be attentive to subtle cues and underlying"
            " emotions. Adapt your communication style to match the user's"
            " preferences and current emotional state. Use tools to persist"
            " information you want to retain in the next conversation. If you"
            " do call tools, all text preceding the tool call is an internal"
            " message. Respond AFTER calling the tool, once you have"
            " confirmation that the tool completed successfully.\n\n",
        ),
        ("placeholder", "{messages}"),
    ]
)

In [20]:


def agent(state: State) -> State:
    """Process the current state and generate a response using the LLM.

    Args:
        state (schemas.State): The current state of the conversation.

    Returns:
        schemas.State: The updated state with the agent's response.
    """
    bound = prompt | llm
    recall_str = (
        "<recall_memory>\n" + "\n".join(state["recall_memories"]) + "\n</recall_memory>"
    )
    # Load the vector store
    print(state["messages"][-1].content)
    vectorstore = Chroma(persist_directory="chroma-BAAI", embedding_function=embedding_model)
    res = vectorstore.similarity_search(state["messages"][-1].content, k=3) 
    context = " ".join([doc.page_content for doc in res])

    prediction = bound.invoke(
        {
            "messages": state["messages"],
            "context": context,
            "recall_memories": recall_str
            
        }
    )
    return {
        "messages": [prediction],
    }


def load_memories(state: State, config: RunnableConfig) -> State:
    """Load memories for the current conversation.

    Args:
        state (schemas.State): The current state of the conversation.
        config (RunnableConfig): The runtime configuration for the agent.

    Returns:
        State: The updated state with loaded memories.
    """
    convo_str = get_buffer_string(state["messages"])
    convo_str = tokenizer.decode(tokenizer.encode(convo_str)[:2048])
    recall_memories = search_recall_memories.invoke(convo_str, config)
    return {
        "recall_memories": recall_memories,
    }


def route_tools(state: State):
    """Determine whether to use tools or end the conversation based on the last message.

    Args:
        state (schemas.State): The current state of the conversation.

    Returns:
        Literal["tools", "__end__"]: The next step in the graph.
    """
    msg = state["messages"][-1]
    if msg.tool_calls:
        return "tools"

    return END

In [21]:
# Create the graph and add nodes
builder = StateGraph(State)
builder.add_node(load_memories)
builder.add_node(agent)
builder.add_node("tools", ToolNode(tools))

# Add edges to the graph
builder.add_edge(START, "load_memories")
builder.add_edge("load_memories", "agent")
builder.add_conditional_edges("agent", route_tools, ["tools", END])
builder.add_edge("tools", "agent")

# Compile the graph
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [22]:
def pretty_print_stream_chunk(chunk):
    for node, updates in chunk.items():
        print(f"Update from node: {node}")
        if "messages" in updates:
            updates["messages"][-1].pretty_print()
        else:
            print(updates)

        print("\n")

In [23]:
# NOTE: we're specifying `user_id` to save memories for a given user
config = {"configurable": {"user_id": "1", "thread_id": "1"}}
i =0
for chunk in graph.stream({"messages": [("user", "Hey I am dheeraj")]}, config=config):
    print (i)
    i=i+1
    print(chunk)


0
{'load_memories': {'recall_memories': []}}
Hey I am dheeraj


C:\Users\dheeraj\AppData\Local\Temp\ipykernel_1928\3863353399.py:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="chroma-BAAI", embedding_function=embedding_model)


1
{'agent': {'messages': [AIMessage(content='<think>\nOkay, so the user just said, "Hey I am dheeraj." I need to respond in a friendly and welcoming manner. Since I don\'t have any prior memory of Dheeraj, I should introduce myself and offer help. I\'ll make sure to keep the tone warm and approachable, maybe ask how I can assist him today. Also, I should remember to save this interaction in the memory tools so I can refer back to it in future conversations. That way, I can provide more personalized support next time he reaches out.\n</think>\n\nHey Dheeraj! Welcome. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 407, 'total_tokens': 537, 'completion_time': 0.472727273, 'prompt_time': 0.072329353, 'queue_time': 0.046950924000000005, 'total_time': 0.545056626}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_2834edf0f4', 'finish_reason': 'stop', 'logprobs': None}, id='run-ca866f86-

In [ ]:
response = graph.stream({"messages": [HumanMessage(content="What is WHO")]}, config=config)
response

TypeError: 'generator' object is not subscriptable

In [35]:
print(response['messages'][0].content)

Hey I am dheeraj


In [41]:
output = list(graph.stream({"messages": [HumanMessage(content="I am dheeraj")]}, config=config))
print(output[-1])  # Now you can safely access the first element


I am dheeraj
{'agent': {'messages': [AIMessage(content='<think>\nAlright, the user has just said, "I am dheeraj." Looking back at the conversation history, I see that they introduced themselves as Dheeraj earlier. So, this might be a way of reaffirming their name or perhaps they\'re starting a new topic.\n\nI should acknowledge their introduction warmly to make them feel welcome. Since they\'ve mentioned their name again, maybe they want to share more about themselves or need assistance with something specific.\n\nI\'ll respond by greeting them and asking how I can assist. Keeping the tone friendly and open-ended should encourage them to share more details about what they need help with.\n</think>\n\nHello, Dheeraj! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 145, 'prompt_tokens': 1404, 'total_tokens': 1549, 'completion_time': 0.527272727, 'prompt_time': 0.075106812, 'queue_time': 0.207020191, 'total_time': 0.602379539}, '

In [50]:
output[-1]['agent']['messages'][0].content

'<think>\nAlright, the user has just said, "I am dheeraj." Looking back at the conversation history, I see that they introduced themselves as Dheeraj earlier. So, this might be a way of reaffirming their name or perhaps they\'re starting a new topic.\n\nI should acknowledge their introduction warmly to make them feel welcome. Since they\'ve mentioned their name again, maybe they want to share more about themselves or need assistance with something specific.\n\nI\'ll respond by greeting them and asking how I can assist. Keeping the tone friendly and open-ended should encourage them to share more details about what they need help with.\n</think>\n\nHello, Dheeraj! How can I assist you today?'

In [27]:
for chunk in graph.stream({"messages": [("user", "What is WHO")]}, config=config):
    pretty_print_stream_chunk(chunk)

Update from node: load_memories
{'recall_memories': []}


What is WHO
Update from node: agent
================================== Ai Message ==================================

<think>

The **World Health Organization (WHO)** is a specialized agency of the United Nations responsible for international public health. It was established on July 22, 1946, and is headquartered in Geneva, Switzerland. The WHO's main objective is to promote health, keep the world safe, and serve all people. It works to achieve its goals through various programs, including global health security, non-communicable diseases, mental health, and promoting health equity.

The WHO is known for its role in setting global health policies, coordinating responses to infectious disease outbreaks (e.g., COVID-19, Ebola), and providing technical assistance to countries to improve their healthcare systems. It also plays a key role in promoting vaccination, eradicating diseases (e.g., smallpox), and addressing global health c

AIMessage(content="<think>\n\n</think>\n\nI only remember what we discuss in our current chat. Each conversation starts fresh to better protect user privacy. Feel free to tell me any preferences or topics you'd like me to focus on - I'll adjust accordingly.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 8, 'total_tokens': 56, 'completion_time': 0.174545455, 'prompt_time': 0.003604036, 'queue_time': 0.047140473, 'total_time': 0.178149491}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_fc872c073e', 'finish_reason': 'stop', 'logprobs': None}, id='run-0e6cbe36-329a-4eb0-9f3b-11290d7d05d2-0', usage_metadata={'input_tokens': 8, 'output_tokens': 48, 'total_tokens': 56})

In [25]:
llm.invoke("Hi ")

AIMessage(content='<think>\n\n</think>\n\nHello! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 6, 'total_tokens': 22, 'completion_time': 0.058181818, 'prompt_time': 0.003487132, 'queue_time': 0.046382137000000004, 'total_time': 0.06166895}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_2834edf0f4', 'finish_reason': 'stop', 'logprobs': None}, id='run-cc49ecb5-cf80-4194-8f23-5be3feb6d74e-0', usage_metadata={'input_tokens': 6, 'output_tokens': 16, 'total_tokens': 22})

In [5]:
response

<Response [200]>

In [7]:
for script in soup(["script", "style"]):
    script.extract()

# Get text content
text = soup.get_text()

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader('https://www.geeksforgeeks.org/machine-learning/')

2025-03-23 15:32:32 - USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
page_data = loader.load()
page_data

In [4]:
user_input ="http://jks"
if user_input.startswith("http://") or user_input.startswith("https://"):
    print("Invalid input")

Invalid input


In [6]:
a = [File(thread_id='78581cd1-df3a-4346-aeaf-30694852b3b1', name='s00466-022-02252-0.pdf', id='e3dab057-e37d-4927-8b1a-fc9ead9643c8', chainlit_key='e3dab057-e37d-4927-8b1a-fc9ead9643c8', url=None, object_key=None, path='G:\NLP\hugging_face_model\.files\edfd1403-1c0d-448a-b814-716e1fe47b9a\e3dab057-e37d-4927-8b1a-fc9ead9643c8.pdf', content=None, display='inline', size=None, for_id='597f2ab1-f578-4675-906a-c79b168fd2a6', language=None, mime='pdf')]

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: malformed \N character escape (413114150.py, line 1)